In [1]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

fp = open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\OoPdfFormExample.pdf', 'rb')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.get_pages(fp)

for page in pages:
    print('Processing next page...')
    interpreter.process_page(page)
    layout = device.get_result()
    for lobj in layout:
        if isinstance(lobj, LTTextBox):
            x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()
            print('At %r is text: %s' % ((x, y), text))

Processing next page...
At (56.8, 675.516) is text: PDF Form Example

At (56.8, 645.8969999999999) is text: This is an example of a user fillable PDF form. Normally PDF is used as a final publishing format. 
However PDF has an option to be used as an entry form that can be edited and saved by the user.

At (56.8, 607.997) is text: The fields of this form have been selected to demonstrate as many as possible of the common 
entry fields.

At (56.8, 569.997) is text: This document and PDF form have been created with OpenOffice (version 3.4.0).

At (56.8, 544.697) is text: To fill out the form, make sure the PDF file is not read-only. If the file is read-only save it first to a 
folder or computer desktop. Close this file and open the saved file.

At (56.8, 506.797) is text: Please fill out the following fields. Important fields are marked yellow.

At (56.8, 468.797) is text: Given Name:

At (56.8, 432.297) is text: Family Name:

At (56.8, 403.397) is text: Address 1:

At (56.8, 379.597000

In [7]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
import pandas as pd
import re

fp = open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\orderform-a.pdf', 'rb')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.get_pages(fp)

In [8]:
k=[]
l=[]
m=[]
for page in pages:
    print('Processing next page...')
    interpreter.process_page(page)
    layout = device.get_result()
    for lobj in layout:
        if isinstance(lobj, LTTextBox):
            x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()
            k.append(x)
            l.append(y)          
            text=re.sub('\n','',text)
            m.append(text)

Processing next page...


In [9]:
df=pd.DataFrame(l,k)
df.reset_index(inplace=True)
df['text']=m
df.rename(columns={'index':'x',0:'y'},inplace=True)

In [10]:
df

,x,y,text
0,37.417,668.885,Order Form
1,452.835,659.014,Order Date:
2,38.835,623.014,Name:
3,38.835,587.014,Address:
4,38.835,551.014,City:
5,38.835,515.014,State:
6,38.835,479.014,ZIP:
7,38.835,443.014,Country:
8,98.008,412.516,Add me to the mailing list
9,37.415,380.407,ShippingShip to:


In [373]:
file=r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\OoPdfFormExample.pdf'

In [389]:
data1=df.copy()

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""Script for working with fillable fields of PDF files

Requires PyPDF2 1.26.0, pandas 0.24

"""
import os
import glob
import argparse
import json
import logging
import pandas as pd
from logging.handlers import RotatingFileHandler

import PyPDF2 as p

logger = logging.getLogger(__name__)
col_list = ['Filename', 'Page', 'Field_name', 'Field_type']


def fillable_pdf(input_file):
    # mode - read field structure
    pdfr = p.PdfFileReader(open(input_file, 'rb'))
    fields_dict = pdfr.getFields() or {}
    res_dict = {}
    for el in fields_dict:
        if fields_dict[el].get('/FT', 'none') == '/Tx':
            res_dict[el] = fields_dict[el].get('/V', '')
        if fields_dict[el].get('/FT', 'none') == '/Btn':
            flags = int(fields_dict[el].get('/Ff', 0))
            flag_radio = (flags >> 15) & 1
            flag_pushbutton = (flags >> 16) & 1
            if not flag_radio and not flag_pushbutton:
                res_dict[el] = {'type': 'checkbox', 'value': fields_dict[el].get('/V', '')}
                lst = list(fields_dict[el].get('/AP', {}).get('/N', {}).keys())
                if lst:
                    res_dict[el]['options'] = lst.copy()
            if flag_radio:
                res_dict[el] = {'type': 'radio_buttons', 'value': fields_dict[el].get('/V', '')}
                lst = set()
                for kid in fields_dict[el].get('/Kids', {}):
                    kid_obj = kid.getObject()
                    lst.update(list(kid_obj.get('/AP', {}).get('/N', {}).keys()))
                if lst:
                    res_dict[el]['options'] = list(lst).copy()
    return res_dict


def parse_pdf_file(input_f):
    rez = []
    fields_dict = fillable_pdf(input_f)
    pdfr = p.PdfFileReader(open(input_f, 'rb'))
    for current_page in range(pdfr.numPages):
        page = pdfr.getPage(current_page)
        if '/Annots' not in page:
            continue
        for j in range(0, len(page['/Annots'])):
            annot = page['/Annots'][j].getObject()
            logger.debug('{} anno:{}'.format(j, annot))
            if '/T' in annot:
                field_name = annot.get('/T')
                if field_name in fields_dict:
                    row = {'Filename': input_f,
                           'Page': current_page+1,
                           'Field_name': field_name,
                           'Field_type': 'text_box'
                           }
                    if isinstance(fields_dict[field_name], dict):
                        row['Field_type'] = fields_dict[field_name]['type']
                    rez.append(row)
            else:
                if annot.get('/Subtype') == '/Widget' and '/Parent' in annot:
                    field_name = annot['/Parent'].getObject().get('/T')
                    if field_name in fields_dict:
                        row = {'Filename': input_f,
                               'Page': current_page+1,
                               'Field_name': field_name,
                               'Field_type': 'text_box'
                               }
                        if isinstance(fields_dict[field_name], dict):
                            row['Field_type'] = fields_dict[field_name]['type']
                        rez.append(row)
    return rez


def parse_pdf_files(input_file_or_folder, output_filename):
    input_data_is_file = False
    if os.path.isfile(input_file_or_folder):
        input_data_is_file = True
    else:
        if not os.path.isdir(input_file_or_folder):
            logger.error('The input data path [{}] is wrong'.format(input_file_or_folder))
            return
    df_total = pd.DataFrame()
    if input_data_is_file:
        logger.info('processing a single file {} ...'.format(input_file_or_folder))
        res = parse_pdf_file(input_file_or_folder)
        if res:
            logger.info('processing {} successful'.format(input_file_or_folder))
        df = pd.DataFrame(res, columns=col_list, dtype=str)
        df_total = df_total.append(df)
        try:
            df_total = df_total[col_list]
            df_total.to_csv(output_filename, index=False)
        except PermissionError:
            logger.error('Error access when writing a file with results [{}]'.format(output_filename))
    else:
        logger.info('processing the whole folder {}'.format(input_file_or_folder))
        for i, ff in enumerate(glob.glob(input_file_or_folder + os.sep + '*.pdf')):
            logger.info('processing {} ...'.format(ff))
            res = parse_pdf_file(ff)
            if res:
                logger.info('processing {} successful'.format(ff))
            else:
                logger.info('skip empty file')
            df = pd.DataFrame(res, columns=col_list, dtype=str)
            df_total = df_total.append(df, sort=False)
        if df_total.shape[1] >= 4:
            df_total = df_total[col_list]
        df_total.to_csv(output_filename, index=False)


def prepare_options():
    parser = argparse.ArgumentParser()
    parser.add_argument('input_file_or_folder', help='input file or folder with pdf files')
    parser.add_argument('-o', nargs='?', default='result.csv', dest='output_filename',
                        help='output file name with results (default - current folder and result.csv)')
    args = parser.parse_args()
    return vars(args)


def setting_logger():
    formatter = logging.Formatter('%(asctime)s-%(name)s-%(levelname)s-%(message)s')
    fh = RotatingFileHandler('fillable_pdf_log.txt', maxBytes=100000, backupCount=1, encoding='utf-8')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s-%(name)s-%(levelname)s-%(message)s',
                        handlers=(fh, ch))



input_file_or_folder=r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Final\OoPdfFormExample.pdf'
output_filename=r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Final\OoPdfFormExample.csv'
parse_pdf_files(input_file_or_folder, output_filename)


def prepare_options():
    parser = argparse.ArgumentParser()
    parser.add_argument('input_file_or_folder', help='input file or folder with pdf files')
    parser.add_argument('-o', nargs='?', default='result.csv', dest='output_filename',
                        help='output file name with results (default - current folder and result.csv)')
    args = parser.parse_args()
    return vars(args)


def setting_logger():
    formatter = logging.Formatter('%(asctime)s-%(name)s-%(levelname)s-%(message)s')
    fh = RotatingFileHandler('fillable_pdf_log.txt', maxBytes=100000, backupCount=1, encoding='utf-8')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s-%(name)s-%(levelname)s-%(message)s',
                        handlers=(fh, ch))

In [5]:
data=pd.read_csv(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\sample_file123.csv')

In [6]:
data

,Filename,Page,Field_name,Field_type
0,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,Name[0],text_box
1,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,Address[0],text_box
2,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,City[0],text_box
3,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,State[0],text_box
4,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,ZIP[0],text_box
5,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,MailingList[0],checkbox
6,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,ShippingMethod[0],radio_buttons
7,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,ShippingMethod[0],radio_buttons
8,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,ShippingMethod[0],radio_buttons
9,C:\Users\murali\Desktop\Srikanth\Project_FONT\...,1,ShipToName[0],text_box


In [11]:
df

,x,y,text
0,37.417,668.885,Order Form
1,452.835,659.014,Order Date:
2,38.835,623.014,Name:
3,38.835,587.014,Address:
4,38.835,551.014,City:
5,38.835,515.014,State:
6,38.835,479.014,ZIP:
7,38.835,443.014,Country:
8,98.008,412.516,Add me to the mailing list
9,37.415,380.407,ShippingShip to:


In [2]:
file=r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\OoPdfFormExample.PDF'

In [3]:
def fillable_pdf(input_file):
    # mode - read field structure
    pdfr = p.PdfFileReader(open(input_file, 'rb'))
    fields_dict = pdfr.getFields() or {}
    res_dict = {}
    for el in fields_dict:
        if fields_dict[el].get('/FT', 'none') == '/Tx':
            res_dict[el] = fields_dict[el].get('/V', '')
        if fields_dict[el].get('/FT', 'none') == '/Btn':
            flags = int(fields_dict[el].get('/Ff', 0))
            flag_radio = (flags >> 15) & 1
            flag_pushbutton = (flags >> 16) & 1
            if not flag_radio and not flag_pushbutton:
                res_dict[el] = {'type': 'checkbox', 'value': fields_dict[el].get('/V', '')}
                lst = list(fields_dict[el].get('/AP', {}).get('/N', {}).keys())
                if lst:
                    res_dict[el]['options'] = lst.copy()
            if flag_radio:
                res_dict[el] = {'type': 'radio_buttons', 'value': fields_dict[el].get('/V', '')}
                lst = set()
                for kid in fields_dict[el].get('/Kids', {}):
                    kid_obj = kid.getObject()
                    lst.update(list(kid_obj.get('/AP', {}).get('/N', {}).keys()))
                if lst:
                    res_dict[el]['options'] = list(lst).copy()
    return res_dict,fields_dict


In [4]:
a,b=fillable_pdf(file)

In [6]:
with open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\file.txt','a') as f:
    print(b, file=f)

In [22]:
with open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\field_details.txt', 'w+') as f:
    for item in a:
        f.write("%s\n" % item)




In [16]:
pdftotree.parse(file, model_path=None, favor_figures=True, visualize=True)

"<html><div id=1><figure bbox=56.69999999999993,56.7,137.39999999999998,538.6></figure><section_header char='P D F   F o r m   E x a m p l e ', top='166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 166.48400000000004 ', left='56.8 67.488 78.992 88.78399999999999 93.264 102.96 112.75199999999998 118.94399999999999 133.23199999999997 137.632 148.32 157.216 166.112 180.304 190.096 194.49599999999998 ', bottom='181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 181.0920000000001 ', right='67.472 79.03999999999999 88.768 93.232 103.039999999999

In [364]:
b='Given Name Text Box'
a='Given Name'
if re.search(a,b):
    print('t')

t


In [58]:
import pandas as pd

In [68]:
data=pd.read_csv(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\final.csv')

In [96]:
k=[]
for i in range(len(df)):    
    p=(df['Field_name'][i],df['Field_type'][i])
    k.append(p)

In [105]:
with open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\checking.txt','a') as f:
    for i in range(len(k)):
        print(k[i], file=f)

In [26]:
!pip install pypdftk


  Stored in directory: C:\Users\murali\AppData\Local\pip\Cache\wheels\b6\9b\68\5ef3c3e0780bc54c02d89e4c8e6cbfc7e9c9cbadbcf021761e
Successfully built pypdftk


In [22]:
from pikepdf import Pdf
pdf = Pdf.open(r'C:\Users\murali\Desktop\Srikanth\Project_FONT\Reference\Main\orderform-a.pdf')

In [30]:
!pip install pdfminer3


  Stored in directory: C:\Users\murali\AppData\Local\pip\Cache\wheels\27\35\61\7d4c0c350683751100186de869681db04d6f2c5ab603eb0510
Successfully built pdfminer3


In [31]:
from argparse import ArgumentParser
import pickle
import pprint
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdftypes import resolve1, PDFObjRef

def load_form(filename):
    """Load pdf form contents into a nested list of name/value tuples"""
    with open(filename, 'rb') as file:
        parser = PDFParser(file)
        doc = PDFDocument()
        parser.set_document(doc)
        doc.set_parser(parser)
        doc.initialize()
        return [load_fields(resolve1(f)) for f in
                   resolve1(doc.catalog['AcroForm'])['Fields']]


ImportError: cannot import name 'PDFDocument'